In [1]:
import pandas as pd
import numpy as np

# read the machine dataset
machines_df = pd.read_csv('machines.csv', sep=';')
# read the surface dataset
surfaces_df = pd.read_csv('surfaces.csv', sep=';')

# define the fitness function to minimize cleaning time
def fitness_function(solution, machines_df, surface_area, surface_time):
    selected_machines = machines_df[solution == 1]
    total_time = np.sum(selected_machines['Time'] * surface_area / selected_machines['productivity'])
    return total_time - surface_time

# define the BPSO algorithm
def bpso(machines_df, surface_df, swarm_size=10, max_iterations=100, c1=2, c2=2, w=0.7):
    # initialize swarm
    swarm = np.random.randint(2, size=(swarm_size, len(machines_df)))
    pbests = swarm.copy()
    fitness_values = np.zeros(swarm_size)
    gbest = swarm[0].copy()
    # initialize fitness values
    for i in range(swarm_size):
        fitness_values[i] = fitness_function(swarm[i], machines_df, surface_df['cleaning area'], surface_df['cleaning time'])
        # update particle bests and global best
        if (fitness_values[i] < fitness_function(gbest, machines_df, surface_df['cleaning area'], surface_df['cleaning time'])).all():
            gbest = swarm[i].copy()
        if (fitness_values[i] < fitness_function(pbests[i], machines_df, surface_df['cleaning area'], surface_df['cleaning time'])).all():
            pbests[i] = swarm[i].copy()
    # run iterations
    for t in range(max_iterations):
        # update swarm
        for i in range(swarm_size):
            r1, r2 = np.random.rand(), np.random.rand()
            vel = w * swarm[i] + c1 * r1 * (pbests[i] - swarm[i]) + c2 * r2 * (gbest - swarm[i])
            swarm[i] = (vel >= 0.5).astype(int)
            # update fitness values
            fitness_values[i] = fitness_function(swarm[i], machines_df, surface_df['cleaning area'], surface_df['cleaning time'])
            # update particle bests and global best
            if (fitness_values[i] < fitness_function(pbests[i], machines_df, surface_df['cleaning area'], surface_df['cleaning time'])).all():
                pbests[i] = swarm[i].copy()
                if (fitness_values[i] < fitness_function(gbest, machines_df, surface_df['cleaning area'], surface_df['cleaning time'])).all():
                    gbest = swarm[i].copy()
    # select machines with ones in gbest
    selected_machines = machines_df[gbest.astype(bool)]
    # calculate total cleaning time
    total_time = (surface_df['cleaning area'] / selected_machines['productivity']).sum()
    return selected_machines, total_time

# run the BPSO algorithm for each surface
for i in range(len(surfaces_df)):
    selected_machines, total_time = bpso(machines_df, surfaces_df.iloc[[i]])
    print(f'Surface {i+1}:')
    print(f'Selected machines: {selected_machines.values}')
    print(f'Total cleaning time: {total_time:.2f} hours\n')


Surface 1:
Selected machines: [['A2' 1 1.5 1575]
 ['A3' 1 2.5 2475]
 ['A4' 1 2.5 1750]
 ['A6' 1 3.5 3150]
 ['A7' 1 2.5 2700]
 ['A10' 1 4.0 3825]
 ['B1' 1 1.5 1400]
 ['B2' 1 2.5 1720]
 ['B4' 1 3.5 2200]
 ['B5' 1 5.0 2200]
 ['C3' 1 3.5 2460]]
Total cleaning time: 0.00 hours

Surface 2:
Selected machines: [['A3' 1 2.5 2475]
 ['A5' 1 3.0 1750]
 ['A6' 1 3.5 3150]
 ['A7' 1 2.5 2700]
 ['A8' 1 2.5 3150]
 ['A10' 1 4.0 3825]
 ['B2' 1 2.5 1720]
 ['B4' 1 3.5 2200]
 ['C1' 1 8.0 5600]
 ['C2' 1 3.0 2970]
 ['C4' 1 4.5 7740]]
Total cleaning time: 0.00 hours

Surface 3:
Selected machines: [['A5' 1 3.0 1750]
 ['A7' 1 2.5 2700]
 ['B1' 1 1.5 1400]
 ['B3' 1 3.5 1720]
 ['B5' 1 5.0 2200]]
Total cleaning time: 0.00 hours

Surface 4:
Selected machines: [['A1' 1 1.0 1225]
 ['A2' 1 1.5 1575]
 ['A5' 1 3.0 1750]
 ['A10' 1 4.0 3825]
 ['B3' 1 3.5 1720]
 ['B4' 1 3.5 2200]
 ['B5' 1 5.0 2200]
 ['C1' 1 8.0 5600]
 ['C4' 1 4.5 7740]]
Total cleaning time: 0.00 hours

Surface 5:
Selected machines: [['A1' 1 1.0 1225]
 ['A2' 1

In [52]:
import numpy as np
import pandas as pd

# Load the dataset of scrubber machines
machines_df = pd.read_csv('machines.csv', sep=';')

# Load the dataset of cleaning sites
sites_df = pd.read_csv('surfaces.csv', sep=';')

# Define the objective function
def objective_function(x):
    total_time = 0
    for i, site in sites_df.iterrows():
        # Calculate the time required to clean the site with the selected scrubber machines
        time = np.sum(x * machines_df['Time'].values * site['cleaning area'] / machines_df['productivity'].values)
        # If the time required is greater than the cleaning time of the site, penalize the solution
        if time > site['cleaning time']:
            total_time += 1000 * (time - site['cleaning time'])
        else:
            total_time += time
    # Return the total time required to clean all the sites
    return total_time

# Define the BPSO algorithm
def bpso(n_particles, n_iterations):
    # Initialize the positions and velocities of the particles
    positions = np.random.randint(2, size=(n_particles, len(machines_df)))
    velocities = np.zeros_like(positions)
    # Initialize the personal best positions and fitness values
    personal_best_positions = positions.copy()
    personal_best_fitness = np.array([objective_function(x) for x in positions])
    # Initialize the global best position and fitness value
    global_best_position = personal_best_positions[personal_best_fitness.argmin()].copy()
    global_best_fitness = personal_best_fitness.min()
    # Iterate over the specified number of iterations
    for i in range(n_iterations):
        # Update the velocities of the particles
        r1, r2 = np.random.uniform(size=(2, n_particles, len(machines_df)))
        velocities = 0.5 * velocities + 1 * r1 * (personal_best_positions - positions) + 1 * r2 * (global_best_position - positions)
        # Apply the sigmoid activation function to the velocities to update the positions
        positions = (1 / (1 + np.exp(-velocities))) > np.random.uniform(size=(n_particles, len(machines_df)))
        # Evaluate the fitness values of the new positions
        fitness = np.array([objective_function(x) for x in positions])
        # Update the personal best positions and fitness values
        improved = fitness < personal_best_fitness
        personal_best_positions[improved] = positions[improved]
        personal_best_fitness[improved] = fitness[improved]
        # Update the global best position and fitness value
        if personal_best_fitness.min() < global_best_fitness:
            global_best_position = personal_best_positions[personal_best_fitness.argmin()].copy()
            global_best_fitness = personal_best_fitness.min()
        # Print the best solution found so far
        print('Iteration {}: Best fitness = {}, Best solution = {}'.format(i, global_best_fitness, global_best_position))

    # Return the best solution found
    return global_best_position

# Run the BPSO algorithm
if __name__ == '__main__':
    # Set the parameters of the BPSO algorithm
    n_particles = 5
    n_iterations = 200
    # Run the BPSO algorithm
    best_solution = bpso(n_particles, n_iterations)
    # Print the optimal combinations of scrubber machines to be deployed to the different sites
    for i, site in sites_df.iterrows():
        machines = machines_df['scrubber'][best_solution * machines_df['productivity'].values * site['cleaning area'] > 0].tolist()
        print('Site {}: {}'.format(i, machines))


Iteration 0: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 1: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 2: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 3: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 4: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 5: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 6: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 7: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 8: Best fitness = 432585.37807888014, Best solution = [0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0]
Iteration 9: Best fitness = 432585.37807888014

In [67]:
import pandas as pd
import numpy as np



# Load data
machines_df = pd.read_csv('machines.csv', delimiter=';')
sites_df = pd.read_csv('surfaces.csv', delimiter=';')

# Define optimization function
def optimize_sites(machine_assignments):
    # Calculate total cleaning time and area for each machine assignment
    total_times = []
    total_areas = []
    for site_idx in range(len(sites_df)):
        assigned_machines = machines_df[machine_assignments[site_idx] == 1]
        total_time = assigned_machines['Time'].sum()
        total_area = assigned_machines['productivity'].sum() * total_time
        total_times.append(total_time)
        total_areas.append(total_area)
        
    # Calculate fitness value
    fitness = 0
    for site_idx in range(len(sites_df)):
        if total_times[site_idx] > sites_df.iloc[site_idx]['cleaning time']:
            # Cleaning time exceeded for this site
            fitness += 1000
        else:
            # Calculate the difference between the target and actual cleaned areas
            diff = abs(total_areas[site_idx] - sites_df.iloc[site_idx]['cleaning area'])
            # Add to fitness
            fitness += diff
    
    return fitness

# Set up binary PSO optimizer
num_sites = len(sites_df)
num_machines = len(machines_df)
options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
bounds = (np.zeros(num_machines * num_sites), np.ones(num_machines * num_sites))
optimizer = BinaryPSO(n_particles=100, dimensions=num_machines * num_sites, options=options, bounds=bounds, init_pos=None)

# Optimize
cost, pos = optimizer.optimize(optimize_sites, iters=1000)

# Extract best machine assignments for each site
machine_assignments = np.reshape(pos, (num_sites, num_machines))
for site_idx in range(len(sites_df)):
    assigned_machines = machines_df[machine_assignments[site_idx] == 1]
    print(f"Site {site_idx+1} - Assigned Machines: {', '.join(assigned_machines['scrubber'].tolist())}")


NameError: name 'BinaryPSO' is not defined